# Task 1 : Topic Modelling Klasifikasi

## Load Library

In [1]:
import csv,numpy,pandas,nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import time

from sklearn.pipeline import Pipeline
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [2]:
#Baca data CSV dengan pandas
df = pandas.read_csv('../data/ds_asg_data.csv')

df.head()

,article_id,article_topic,article_content
0,93205794,Internasional,Kepolisian Inggris tengah memburu pelaku yang...
1,93186698,Ekonomi,Seluruh layanan transaksi di jalan tol akan m...
2,93191463,Teknologi,"\nHari ini, Rabu (23/8), ternyata menjadi har..."
3,93219292,Ekonomi,Saat ini Indonesia hanya memiliki cadangan ba...
4,343106,Hiburan,"Hari ini, Selasa (1/8), pedangdut Ridho Rhoma..."


In [3]:
#View berdasar topic
groupby= df.groupby('article_topic').size()
print (groupby)

article_topic
Bisnis             25
Bojonegoro        260
Ekonomi          1762
Haji             1497
Health            131
Hiburan          1466
Horor              50
Hukum              85
Internasional     741
Jakarta            12
K-Pop              61
KPK                37
Kesehatan         195
Keuangan           14
Lifestyle         572
MotoGP             35
Obat-obatan        58
Otomotif          174
Pendidikan         70
Personal           81
Pilgub Jatim       25
Politik           104
Regional           35
Sains             174
Sejarah            70
Sepak Bola       1184
Sports            435
Teknologi         571
Travel             76
dtype: int64


View berdasarkan topik dan didapatkan 29 topik dengan topik Ekonomi mempunyai proporsi paling banyak, dan dapat dilihat data tersebut tidak proporsional

Dicek dahulu apakah data terdapat missing value

In [4]:
#Cek data dan normalisasi
#cek bila ada yang kosong
cek_bentuk= df['article_content'].shape
cek_null=df.isnull()
cek_jumlah_null=df.isnull().sum()

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(10000,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         

Setelah cek data apakah ada data yang null atau tidak, karena ada data yang null/missing value maka perlu dinormalisasi terlebih dahulu.

Kita hapus data yang terdapat missing value

In [5]:
#Drop/Hapus data yang kosong
modifiedData = df.dropna()

cek_bentuk= modifiedData['article_content'].shape
cek_null=modifiedData.isnull()
cek_jumlah_null=modifiedData.isnull().sum()

#Save Data yang sudah dimodifikasi ->opsional
modifiedData.to_csv('modifiedData.csv',index=False)

print (cek_bentuk)
print (cek_null)
print (cek_jumlah_null)

(9964,)
      article_id  article_topic  article_content
0          False          False            False
1          False          False            False
2          False          False            False
3          False          False            False
4          False          False            False
5          False          False            False
6          False          False            False
7          False          False            False
8          False          False            False
9          False          False            False
10         False          False            False
11         False          False            False
12         False          False            False
13         False          False            False
14         False          False            False
15         False          False            False
16         False          False            False
17         False          False            False
18         False          False            False
19         F

Kita Hapus data yang missing value, maka 36 data yang kosong tersebut dihilangkan. Lalu kita load lagi data yang telah dinormalisasi.

In [6]:
#Baca data CSV dengan pandas
df = pandas.read_csv('modifiedData.csv')
Id=df['article_id']
Con= df['article_content']
Topic = df['article_topic']

Setelah itu dilakukan tahap tokenisasi yaitu memecah kalimat menjadi kata-kata yang dilakukan untuk menjadikan sebuah kalimat menjadi lebih bermakna.

In [7]:
#Pisahkan kata/tokenize data

PK= [nltk.word_tokenize(PisahKata) for PisahKata in df['article_content']]

Setelah itu masuk ketahap stopword removal/filtrasi yaitu filtrasi adalah tahap mengambil kata-kata penting

In [8]:
stopword = ['ada','adalah','adanya','adapun','agak','agaknya','agar','akan','akankah',
                 'akhir','akhiri','akhirnya','aku','akulah','amat','amatlah','anda','andalah',
                 'antar','antara','antaranya','apa','apaan','apabila','apakah','apalagi','apatah',
                 'artinya','asal','asalkan','atas','atau','ataukah','ataupun','awal','awalnya','bagai',
                 'bagaikan','bagaimana','bagaimanakah','bagaimanapun','bagi','bagian','bahkan','bahwa',
                 'bahwasanya','baik','bakal','bakalan','balik','banyak','bapak','baru','bawah','beberapa',
                 'begini','beginian','beginikah','beginilah','begitu','begitukah','begitulah','begitupun',
                 'bekerja','belakang','belakangan','belum','belumlah','benar','benarkah','benarlah','berada',
                 'berakhir','berakhirlah','berakhirnya','berapa','berapakah','berapalah','berapapun','berarti',
                 'berawal','berbagai','berdatangan','beri','berikan','berikut','berikutnya','berjumlah','berkali-kali',
                 'berkata','berkehendak','berkeinginan','berkenaan','berlainan','berlalu','berlangsung','berlebihan','bermacam',
                 'bermacam-macam','bermaksud','bermula','bersama','bersama-sama','bersiap','bersiap-siap','bertanya','bertanya-tanya','berturut','berturut-turut','bertutur','berujar','berupa','besar','betul','betulkah','biasa','biasanya','bila','bilakah','bisa','bisakah','boleh','bolehkah','bolehlah','buat','bukan','bukankah','bukanlah','bukannya','bulan','bung','cara','caranya','cukup','cukupkah','cukuplah','cuma','dahulu','dalam','dan','dapat','dari','daripada','datang','dekat','demi','demikian','demikianlah','dengan','depan','di','dia','diakhiri','diakhirinya','dialah','diantara','diantaranya','diberi','diberikan','diberikannya','dibuat','dibuatnya','didapat','didatangkan','digunakan','diibaratkan','diibaratkannya','diingat','diingatkan','diinginkan','dijawab','dijelaskan','dijelaskannya','dikarenakan','dikatakan','dikatakannya','dikerjakan','diketahui','diketahuinya','dikira','dilakukan','dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya','dimaksudnya','diminta','dimintai','dimisalkan','dimulai','dimulailah','dimulainya','dimungkinkan','dini','dipastikan','diperbuat','diperbuatnya','dipergunakan','diperkirakan','diperlihatkan','diperlukan','diperlukannya','dipersoalkan','dipertanyakan','dipunyai','diri','dirinya','disampaikan','disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan','ditandaskan','ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan','ditunjukkannya','ditunjuknya','dituturkan','dituturkannya','diucapkan','diucapkannya','diungkapkan','dong','dua','dulu','empat','enggak','enggaknya','entah','entahlah','guna','gunakan','hal','hampir','hanya','hanyalah','hari','harus','haruslah','harusnya','hendak','hendaklah','hendaknya','hingga','ia','ialah','ibarat','ibaratkan','ibaratnya','ibu','ikut','ingat','ingat-ingat','ingin','inginkah','inginkan','ini','inikah','inilah','itu','itukah','itulah','jadi','jadilah','jadinya','jangan','jangankan','janganlah','jauh','jawab','jawaban','jawabnya','jelas','jelaskan','jelaslah','jelasnya','jika','jikalau','juga','jumlah','jumlahnya','justru','kala','kalau','kalaulah','kalaupun','kalian','kami','kamilah','kamu','kamulah','kan','kapan','kapankah','kapanpun','karena','karenanya','kasus','kata','katakan','katakanlah','katanya','ke','keadaan','kebetulan','kecil','kedua','keduanya','keinginan','kelamaan','kelihatan','kelihatannya','kelima','keluar','kembali','kemudian','kemungkinan','kemungkinannya','kenapa','kepada','kepadanya','kesampaian','keseluruhan','keseluruhannya','keterlaluan','ketika','khususnya','kini','kinilah','kira','kira-kira','kiranya','kita','kitalah','kok','kurang','lagi','lagian','lah','lain','lainnya','lalu','lama','lamanya','lanjut','lanjutnya','lebih','lewat','lima','luar','macam','maka','makanya','makin','malah','malahan','mampu','mampukah','mana','manakala','manalagi','masa','masalah','masalahnya','masih','masihkah','masing','masing-masing','mau','maupun','melainkan','melakukan','melalui','melihat','melihatnya','memang','memastikan','memberi','memberikan','membuat','memerlukan','memihak','meminta','memintakan','memisalkan','memperbuat','mempergunakan','memperkirakan','memperlihatkan','mempersiapkan','mempersoalkan','mempertanyakan','mempunyai','memulai','memungkinkan','menaiki','menambahkan','menandaskan','menanti','menanti-nanti','menantikan','menanya','menanyai','menanyakan','mendapat','mendapatkan','mendatang','mendatangi','mendatangkan','menegaskan','mengakhiri','mengapa','mengatakan','mengatakannya','mengenai','mengerjakan','mengetahui','menggunakan','menghendaki','mengibaratkan','mengibaratkannya','mengingat','mengingatkan','menginginkan','mengira','mengucapkan','mengucapkannya','mengungkapkan','menjadi','menjawab','menjelaskan','menuju','menunjuk','menunjuki','menunjukkan','menunjuknya','menurut','menuturkan','menyampaikan','menyangkut','menyatakan','menyebutkan','menyeluruh','menyiapkan','merasa','mereka','merekalah','merupakan','meski','meskipun','meyakini','meyakinkan','minta','mirip','misal','misalkan','misalnya','mula','mulai','mulailah','mulanya','mungkin','mungkinkah','nah','naik','namun','nanti','nantinya','nyaris','nyatanya','oleh','olehnya','pada','padahal','padanya','pak','paling','panjang','pantas','para','pasti','pastilah','penting','pentingnya','per','percuma','perlu','perlukah','perlunya','pernah','persoalan','pertama','pertama-tama','pertanyaan','pertanyakan','pihak','pihaknya','pukul','pula','pun','punya','rasa','rasanya','rata','rupanya','saat','saatnya','saja','sajalah','saling','sama','sama-sama','sambil','sampai','sampai-sampai','sampaikan','sana','sangat','sangatlah','satu','saya','sayalah','se','sebab','sebabnya','sebagai','sebagaimana','sebagainya','sebagian','sebaik','sebaik-baiknya','sebaiknya','sebaliknya','sebanyak','sebegini','sebegitu','sebelum','sebelumnya','sebenarnya','seberapa','sebesar','sebetulnya','sebisanya','sebuah','sebut','sebutlah','sebutnya','secara','secukupnya','sedang','sedangkan','sedemikian','sedikit','sedikitnya','seenaknya','segala','segalanya','segera','seharusnya','sehingga','seingat','sejak','sejauh','sejenak','sejumlah','sekadar','sekadarnya','sekali','sekali-kali','sekalian','sekaligus','sekalipun','sekarang','sekarang','sekecil','seketika','sekiranya','sekitar','sekitarnya','sekurang-kurangnya','sekurangnya','sela','selain','selaku','selalu','selama','selama-lamanya','selamanya','selanjutnya','seluruh','seluruhnya','semacam','semakin','semampu','semampunya','semasa','semasih','semata','semata-mata','semaunya','sementara','semisal','semisalnya','sempat','semua','semuanya','semula','sendiri','sendirian','sendirinya','seolah','seolah-olah','seorang','sepanjang','sepantasnya','sepantasnyalah','seperlunya','seperti','sepertinya','sepihak','sering','seringnya','serta','serupa','sesaat','sesama','sesampai','sesegera','sesekali','seseorang','sesuatu','sesuatunya','sesudah','sesudahnya','setelah','setempat','setengah','seterusnya','setiap','setiba','setibanya','setidak-tidaknya','setidaknya','setinggi','seusai','sewaktu','siap','siapa','siapakah','siapapun','sini','sinilah','soal','soalnya','suatu','sudah','sudahkah','sudahlah','supaya','tadi','tadinya','tahu','tahun','tak','tambah','tambahnya','tampak','tampaknya','tandas','tandasnya','tanpa','tanya','tanyakan','tanyanya','tapi','tegas','tegasnya','telah','tempat','tengah','tentang','tentu','tentulah','tentunya','tepat','terakhir','terasa','terbanyak','terdahulu','terdapat','terdiri','terhadap','terhadapnya','teringat','teringat-ingat','terjadi','terjadilah','terjadinya','terkira','terlalu','terlebih','terlihat','termasuk','ternyata','tersampaikan','tersebut','tersebutlah','tertentu','tertuju','terus','terutama','tetap','tetapi','tiap','tiba','tiba-tiba','tidak','tidakkah','tidaklah','tiga','tinggi','toh','tunjuk','turut','tutur','tuturnya','ucap','ucapnya','ujar','ujarnya','umum','umumnya','ungkap','ungkapnya','untuk','usah','usai','waduh','wah','wahai','waktu','waktunya','walau','walaupun','wong','yaitu','yakin','yakni','yang']

sw=[]

for conten in PK:
    kata= list(filter(lambda x: x not in stopword, conten))
    sw.append(kata)

df['stopword'] = sw

Setelah menghilangkan kata-kata yang tidak penting akan tersisa kata-kata yang penting saja pada data. lalu kita ubah data dari list ke string untuk proses selanjutnya.

In [9]:
#Ubah list ke str
string = map(' '.join, sw)

## Feature Extraction

Lalu setelah itu masuk ketahap feature extraction dimana kata-kata pada data diubah menjadi vector model menggunakan term frequency(tf) dan term frequency-inverse document frequency(tf-idf). tf(countvector/raw count) cocok digunakan untuk algoritma naive bayes karena naive bayes menggunakan probabilitas dalam prediksinya, dan tf-idf cocok untuk algoritma SVM karena TF-IDF mengukur relevansi, bukan frekuensi. Artinya, jumlah kata digantikan dengan nilai TF-IDF di seluruh dataset. 

In [10]:
#CountVector/raw count
count_vect = CountVectorizer()
count = count_vect.fit_transform(string)

#tf-idf
tf_transformer = TfidfTransformer().fit(count)
X_train_tf = tf_transformer.transform(count)
print (X_train_tf)

  (0, 41690)	0.31868739085265324
  (0, 35543)	0.22343718754799582
  (0, 51457)	0.086947941551807
  (0, 63715)	0.12045952256960622
  (0, 50680)	0.10840495775857466
  (0, 14912)	0.24480796826896137
  (0, 79938)	0.14630997870732254
  (0, 47412)	0.282032599471728
  (0, 75492)	0.1780299110284095
  (0, 1900)	0.09621245926144197
  (0, 61618)	0.11591891954000397
  (0, 84615)	0.15496105386747977
  (0, 66447)	0.12090168105148284
  (0, 38301)	0.044296161657492485
  (0, 62267)	0.051302804673997275
  (0, 77779)	0.0743893006264868
  (0, 871)	0.04186524312591781
  (0, 54970)	0.07703333079153188
  (0, 8909)	0.07607253273919327
  (0, 43737)	0.08601740271837825
  (0, 49373)	0.08719043687285524
  (0, 72985)	0.1251249224240002
  (0, 22658)	0.05435663377818881
  (0, 85339)	0.04420135341259307
  (0, 31605)	0.08176857179202043
  :	:
  (9963, 9314)	0.029212684520568473
  (9963, 27774)	0.03679745339177746
  (9963, 54200)	0.02939892541579596
  (9963, 27484)	0.033126867804607774
  (9963, 2614)	0.1067281271952591

Hasil diatas adalah hasil pembobotan tiap kata pada data, pembobotan menghitung seberapa pentingnya sebuah kata dalam sebuah
dokumen.

## Modelling and Evaluasi pada data sendiri

Dilakukan prediksi dengan langsung melakukan tes pada data train itu sendiri, dalam model kali ini menggunakan algoritma naive bayes dan juga support vector machine.

Kita coba dengan Naive Bayes

In [11]:
#Predictor Naive Bayes & Support Vector Machine()
topic_modelling = MultinomialNB().fit(count, Topic)

prediksi_semua = topic_modelling.predict(count)

print ('accuracy', accuracy_score(Topic, prediksi_semua))
print ('(row=expected, col=predicted)')
print (classification_report(Topic, prediksi_semua))

accuracy 0.9139903653151344
(row=expected, col=predicted)
               precision    recall  f1-score   support

       Bisnis       1.00      0.16      0.28        25
   Bojonegoro       0.95      0.95      0.95       260
      Ekonomi       0.92      0.99      0.95      1760
         Haji       0.99      0.99      0.99      1497
       Health       0.84      0.83      0.84       131
      Hiburan       0.91      0.98      0.94      1448
        Horor       1.00      0.68      0.81        50
        Hukum       0.96      0.86      0.91        85
Internasional       0.89      0.97      0.93       739
      Jakarta       0.00      0.00      0.00        12
        K-Pop       1.00      0.28      0.44        61
          KPK       1.00      0.76      0.86        37
    Kesehatan       0.80      0.93      0.86       195
     Keuangan       1.00      0.07      0.13        14
    Lifestyle       0.84      0.94      0.88       568
       MotoGP       1.00      0.31      0.48        35
  Obat

c:\users\user\appdata\local\programs\python\python35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Hasil dengan Naive Bayes dengan tingkat akurasi 0.91

kemudian coba dengan SVM

In [12]:
#Predictor Naive Bayes BernoulliNB() & SVM LinearSVC()
topic_modelling = LinearSVC().fit(X_train_tf, Topic)

prediksi_semua = topic_modelling.predict(X_train_tf)

print ('accuracy', accuracy_score(Topic, prediksi_semua))

print (classification_report(Topic, prediksi_semua))

accuracy 0.9956844640706544
               precision    recall  f1-score   support

       Bisnis       1.00      0.96      0.98        25
   Bojonegoro       1.00      1.00      1.00       260
      Ekonomi       1.00      1.00      1.00      1760
         Haji       1.00      1.00      1.00      1497
       Health       0.99      0.98      0.98       131
      Hiburan       0.99      1.00      0.99      1448
        Horor       1.00      1.00      1.00        50
        Hukum       1.00      1.00      1.00        85
Internasional       1.00      1.00      1.00       739
      Jakarta       1.00      1.00      1.00        12
        K-Pop       1.00      0.92      0.96        61
          KPK       1.00      1.00      1.00        37
    Kesehatan       0.98      0.97      0.98       195
     Keuangan       1.00      1.00      1.00        14
    Lifestyle       1.00      1.00      1.00       568
       MotoGP       1.00      1.00      1.00        35
  Obat-obatan       0.98      0.98  

Hasil dengan SVM dengan tingkat akurasi 0.995

Pada evaluasi diatas dilakukan klasifikasi pada data yang sama yang digunakan untuk data latih. Hasil dari klasifikasi tersebut tidaklah valid karena model yang diklasifikasikan akan terjadi overfitting dimana kondisi dimana sebuah model klasifikasi dapat melakukan klasifikasi yang sangat baik terhadap set data latih namun sangat buruk saat melakukan klasifikasi data yang baru atau yang belum pernah ada.

## Modelling and Evaluasi dengan Cross Validation

Maka dari itu agar tidak terjadi overfitting klasifikasi dilakukan dengan data yang berbeda dari data latih, salah satu caranya adalah dengan K-Fold Cross Validation. Dengan K-Fold Cross Validation data latih tersebut dibagi menjadi data latih dan data tes oleh sistem sehingga data yang digunakan dalam klasifikasi merupakan data baru dan dapat terhindar dari overfitting.

Coba dengan Naive Bayes Classifier

In [13]:
#Performing klasifikasi dgn cross validation
#Naive Bayes Classifier & SVM

model = MultinomialNB()

scores = cross_val_score(model,  
                         count,  
                         Topic,  
                         cv=StratifiedKFold(Topic, n_folds=10),    
                         scoring='f1_micro',  
                         n_jobs=-1,
                         )
                         
print (scores)

print (scores.mean(), scores.std())

[0.8343254  0.8278607  0.83964143 0.81536926 0.844      0.83282981
 0.84072581 0.84327604 0.84584178 0.84162437]
0.8365494591067536 0.008847254962295797


kemudian kita coba dengan SVM

In [14]:
#Performing klasifikasi dgn cross validation
#Naive Bayes Classifier & SVM

model = LinearSVC()

scores = cross_val_score(model,  
                         X_train_tf,  
                         Topic,  
                         cv=StratifiedKFold(Topic, n_folds=10),    
                         scoring='f1_micro',  
                         n_jobs=-1,
                         )
                         
print (scores)

print (scores.mean(), scores.std())

[0.88095238 0.88059701 0.9063745  0.88023952 0.9        0.88519637
 0.87701613 0.88372093 0.89756592 0.89035533]
0.8882018105585174 0.009432835421666515


Didapatkan hasil bahwa perform algoritma Support Vector Machine lebih bagus dalam membuat prediksi pada data tersebut.

## Future Works
- Menggunakan fungsi regex untuk dapat mengklasifikasikan berdasarkan kata-kata kunci yang termasuk dalam topik yang mana
- Menggunakan algoritma machine learning dan mecoba menerapkan deep learning dalam prediksi
- Membuat normalisasi pada data yang belum proporsional
- Mencoba menambah proses yang diperlukan pada preprocessing data seperti stemming, dan lain sebagainya
- Mencoba Tuning parameter pada countvectorizer, tf-idf dan algoritma machine learning